In [1]:
import pandas as pd 

from utils.assigner import RandomAssigner, GreedyAssigner
from utils.evaluation import (
    calc_pair_frequencies, 
    evaluate_pair_fairness, 
    count_pair_frequency_distribution
)

In [2]:
total_participants = 15
total_rooms = 3
total_rounds = 3

random_assigner = RandomAssigner(total_participants, total_rooms, total_rounds)
greedy_assigner = GreedyAssigner(total_participants, total_rooms, total_rounds)

random_assignments = random_assigner.generate_assignments()
greedy_assignments = greedy_assigner.generate_assignments()

print(random_assignments)
print(greedy_assignments)

[defaultdict(<class 'list'>, {1: [2, 11, 10, 6, 12], 2: [3, 4, 8, 9, 5], 3: [1, 15, 13, 7, 14]}), defaultdict(<class 'list'>, {1: [6, 14, 1, 4, 3], 2: [2, 8, 15, 13, 5], 3: [11, 7, 12, 10, 9]}), defaultdict(<class 'list'>, {1: [14, 10, 12, 6, 1], 2: [3, 7, 15, 13, 4], 3: [11, 5, 9, 8, 2]})]
[defaultdict(<class 'list'>, {1: [13, 5, 4, 10, 3], 2: [9, 15, 6, 8, 1], 3: [11, 14, 7, 12, 2]}), defaultdict(<class 'list'>, {1: [7, 1, 5, 12, 15], 2: [11, 6, 10, 14, 8], 3: [2, 9, 3, 13, 4]}), defaultdict(<class 'list'>, {1: [3, 15, 14, 7, 8], 2: [1, 10, 2, 5, 11], 3: [4, 6, 12, 13, 9]})]


In [3]:
count_pair_frequency_distribution(calc_pair_frequencies(random_assignments))

Counter({1: 47, 2: 14, 3: 5})

In [4]:
count_pair_frequency_distribution(calc_pair_frequencies(greedy_assignments))

Counter({1: 53, 2: 17, 3: 1})

In [5]:
evaluate_pair_fairness(calc_pair_frequencies(random_assignments))

{'mean': np.float64(1.3636363636363635),
 'std_dev': np.float64(0.6188053895958223)}

In [6]:
evaluate_pair_fairness(calc_pair_frequencies(greedy_assignments))

{'mean': np.float64(1.267605633802817),
 'std_dev': np.float64(0.47345736095694685)}

In [7]:
random_res = []
greedy_res = []

for _ in range(1000):
    random_assignments = random_assigner.generate_assignments()
    greedy_assignments = greedy_assigner.generate_assignments()

    random_pair_freq = calc_pair_frequencies(random_assignments)
    greedy_pair_freq = calc_pair_frequencies(greedy_assignments)

    random_freq_dist = count_pair_frequency_distribution(random_pair_freq)
    greedy_freq_dist = count_pair_frequency_distribution(greedy_pair_freq)

    random_fairness = evaluate_pair_fairness(random_pair_freq)
    greedy_fairness = evaluate_pair_fairness(greedy_pair_freq)

    random_fairness["onepair"] = random_freq_dist.get(1)
    greedy_fairness["onepair"] = greedy_freq_dist.get(1)

    random_res.append(random_fairness)
    greedy_res.append(greedy_fairness)


In [8]:
random_df = pd.DataFrame(random_res)
greedy_df = pd.DataFrame(greedy_res)

# 各指標について行ごとに比較
comparison_results = {
    'mean': (random_df['mean'] > greedy_df['mean']).value_counts(),
    'std_dev': (random_df['std_dev'] > greedy_df['std_dev']).value_counts(),
    'onepair': (random_df['onepair'] < greedy_df['onepair']).value_counts()
}

print("各指標についてrandom vs greedyの勝敗カウント:")
for metric, results in comparison_results.items():
    print(f"\n{metric}:")
    random_wins = results.get(False, 0)
    greedy_wins = results.get(True, 0)
    total = random_wins + greedy_wins
    print(f"random の勝ち: {random_wins} ({random_wins/total*100:.1f}%)")
    print(f"greedy の勝ち: {greedy_wins} ({greedy_wins/total*100:.1f}%)")


各指標についてrandom vs greedyの勝敗カウント:

mean:
random の勝ち: 147 (14.7%)
greedy の勝ち: 853 (85.3%)

std_dev:
random の勝ち: 121 (12.1%)
greedy の勝ち: 879 (87.9%)

onepair:
random の勝ち: 179 (17.9%)
greedy の勝ち: 821 (82.1%)
